In [76]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [74]:
!pip install pyalgotrading

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
!pip install pandas
!pip install alpha_vantage
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3099, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
Attribut

In [77]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import plotly.graph_objects as go

class ScriptData:
    def __init__(self, api_key):
        self.api_key = api_key
        self.ts = TimeSeries(key=self.api_key, output_format='pandas')
        self.data = {}

    def fetch_intraday_data(self, script):
        data, _ = self.ts.get_intraday(symbol=script, interval='1min', outputsize='full')
        self.data[script] = data

    def convert_intraday_data(self, script):
        data = self.data.get(script)
        if data is None:
            print(f"No intraday data available for {script}")
            return
        data = data.rename(columns={
            '1. open': 'open',
            '2. high': 'high',
            '3. low': 'low',
            '4. close': 'close',
            '5. volume': 'volume'
        })
        data.index = pd.to_datetime(data.index)
        self.data[script] = data[['open', 'high', 'low', 'close', 'volume']]

    def __getitem__(self, key):
        return self.data.get(key)

def indicator1(df, timeperiod):
    indicator_df = pd.DataFrame()
    indicator_df['timestamp'] = df.index
    indicator_df['indicator'] = df['close'].rolling(window=timeperiod).mean()
    return indicator_df

class Strategy:
    def __init__(self, script):
        self.script = script
        self.script_data = ScriptData(api_key)
        self.script_data.fetch_intraday_data(script)
        self.script_data.convert_intraday_data(script)
        self.indicator_data = indicator1(self.script_data[script], timeperiod=5)
        self.signals = self.generate_signals()

    def get_script_data(self):
        print(f"Intraday data for {self.script}:")
        print(self.script_data[self.script])

    def get_signals(self):
        print(f"Signals for {self.script}:")
        print(self.signals)

    def generate_signals(self):
        signals = pd.DataFrame()
        signals['timestamp'] = self.script_data[self.script].index
        signals['signal'] = 'NO_SIGNAL'


        signals['cut_direction'] = 'NO_CUT'
        for i in range(1, len(self.indicator_data)):
            if self.indicator_data['indicator'][i] > self.script_data[self.script]['close'][i] and \
                    self.indicator_data['indicator'][i - 1] < self.script_data[self.script]['close'][i - 1]:
                signals.loc[i, 'cut_direction'] = 'CUT_UPWARDS'
            elif self.indicator_data['indicator'][i] < self.script_data[self.script]['close'][i] and \
                    self.indicator_data['indicator'][i - 1] > self.script_data[self.script]['close'][i - 1]:
                signals.loc[i, 'cut_direction'] = 'CUT_DOWNWARDS'

        filtered_signals = signals[signals['signal'].isin(['BUY', 'SELL'])]
        return signals


api_key = 'WHS8DRK1NBGUSVJM'
script_data = ScriptData(api_key)

script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
print("Intraday data for GOOGLE:")
print(script_data['GOOGL'])

script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')
print("Intraday data for APPLE:")
print(script_data['AAPL'])

script_data.fetch_intraday_data('NVDA')
script_data.convert_intraday_data('NVDA')
print("Intraday data for NVIDIA:")
print(script_data['NVDA'])

Intraday data for GOOGLE:
                       open    high     low   close  volume
date                                                       
2023-06-16 19:59:00  123.65  123.65  123.65  123.65   192.0
2023-06-16 19:58:00  123.60  123.60  123.60  123.60   377.0
2023-06-16 19:57:00  123.63  123.63  123.60  123.60   950.0
2023-06-16 19:53:00  123.71  123.71  123.70  123.70   611.0
2023-06-16 19:51:00  123.75  123.76  123.75  123.76   684.0
...                     ...     ...     ...     ...     ...
2023-06-05 04:50:00  124.14  124.14  124.14  124.14   490.0
2023-06-05 04:49:00  124.16  124.16  124.16  124.16   153.0
2023-06-05 04:05:00  124.09  124.09  124.09  124.09   215.0
2023-06-05 04:04:00  124.12  124.12  124.12  124.12   283.0
2023-06-05 04:00:00  124.71  124.71  124.71  124.71  1085.0

[6260 rows x 5 columns]
Intraday data for APPLE:
                       open    high     low   close  volume
date                                                       
2023-06-16 19:59:00  184

In [61]:
indicator1(script_data['AAPL'], timeperiod=5)


,timestamp,indicator
0,2023-06-16 19:59:00,NaN
1,2023-06-16 19:58:00,NaN
2,2023-06-16 19:51:00,NaN
3,2023-06-16 19:49:00,NaN
4,2023-06-16 19:48:00,NaN
...,...,...
7703,2023-06-05 04:04:00,NaN
7704,2023-06-05 04:03:00,NaN
7705,2023-06-05 04:02:00,NaN
7706,2023-06-05 04:01:00,NaN


In [62]:
indicator1(script_data['GOOGL'], timeperiod=5)


,timestamp,indicator
0,2023-06-16 19:59:00,NaN
1,2023-06-16 19:58:00,NaN
2,2023-06-16 19:57:00,NaN
3,2023-06-16 19:53:00,NaN
4,2023-06-16 19:51:00,NaN
...,...,...
6255,2023-06-05 04:50:00,NaN
6256,2023-06-05 04:49:00,NaN
6257,2023-06-05 04:05:00,NaN
6258,2023-06-05 04:04:00,NaN


In [44]:
indicator1(script_data['NVDA'], timeperiod=5)

,timestamp,indicator
0,2023-06-16 19:59:00,NaN
1,2023-06-16 19:58:00,NaN
2,2023-06-16 19:57:00,NaN
3,2023-06-16 19:55:00,NaN
4,2023-06-16 19:54:00,NaN
...,...,...
8071,2023-06-05 04:04:00,NaN
8072,2023-06-05 04:03:00,NaN
8073,2023-06-05 04:02:00,NaN
8074,2023-06-05 04:01:00,NaN


In [83]:
# Strategy for GOOGLE
strategy = Strategy('GOOGL')
strategy.get_script_data()
strategy.get_signals()



Intraday data for GOOGL:
                       open    high     low   close  volume
date                                                       
2023-06-16 19:59:00  123.65  123.65  123.65  123.65   192.0
2023-06-16 19:58:00  123.60  123.60  123.60  123.60   377.0
2023-06-16 19:57:00  123.63  123.63  123.60  123.60   950.0
2023-06-16 19:53:00  123.71  123.71  123.70  123.70   611.0
2023-06-16 19:51:00  123.75  123.76  123.75  123.76   684.0
...                     ...     ...     ...     ...     ...
2023-06-05 04:50:00  124.14  124.14  124.14  124.14   490.0
2023-06-05 04:49:00  124.16  124.16  124.16  124.16   153.0
2023-06-05 04:05:00  124.09  124.09  124.09  124.09   215.0
2023-06-05 04:04:00  124.12  124.12  124.12  124.12   283.0
2023-06-05 04:00:00  124.71  124.71  124.71  124.71  1085.0

[6260 rows x 5 columns]
Signals for GOOGL:
               timestamp     signal cut_direction
0    2023-06-16 19:59:00  NO_SIGNAL        NO_CUT
1    2023-06-16 19:58:00  NO_SIGNAL        NO_CUT
2

In [82]:
# Strategy for APPLE
strategy = Strategy('AAPL')
strategy.get_script_data()
strategy.get_signals()

Intraday data for AAPL:
                       open    high     low   close  volume
date                                                       
2023-06-16 19:59:00  184.95  184.95  184.94  184.94   336.0
2023-06-16 19:58:00  184.95  184.95  184.95  184.95   273.0
2023-06-16 19:51:00  184.99  184.99  184.99  184.99   333.0
2023-06-16 19:49:00  184.99  185.00  184.99  184.99   757.0
2023-06-16 19:48:00  185.00  185.00  185.00  185.00   487.0
...                     ...     ...     ...     ...     ...
2023-06-05 04:04:00  181.32  181.44  181.32  181.44  1215.0
2023-06-05 04:03:00  181.24  181.32  181.24  181.32  1000.0
2023-06-05 04:02:00  181.26  181.38  181.21  181.38  1769.0
2023-06-05 04:01:00  181.31  181.35  181.22  181.23  3633.0
2023-06-05 04:00:00  181.22  182.00  181.11  181.42  6750.0

[7708 rows x 5 columns]
Signals for AAPL:
               timestamp     signal cut_direction
0    2023-06-16 19:59:00  NO_SIGNAL        NO_CUT
1    2023-06-16 19:58:00  NO_SIGNAL        NO_CUT
2  

In [81]:
# Strategy for NVIDIA
strategy = Strategy('NVDA')
strategy.get_script_data()
strategy.get_signals()

Intraday data for NVDA:
                         open      high       low     close  volume
date                                                               
2023-06-16 19:59:00  427.6000  427.6500  427.6000  427.6200  2399.0
2023-06-16 19:58:00  427.6600  427.7000  427.6000  427.6100  3843.0
2023-06-16 19:57:00  427.6700  427.7500  427.6700  427.7500  2583.0
2023-06-16 19:55:00  427.7000  427.7000  427.7000  427.7000   336.0
2023-06-16 19:54:00  427.8800  427.8800  427.6000  427.6000  4014.0
...                       ...       ...       ...       ...     ...
2023-06-05 04:04:00  391.0583  391.0583  390.9583  390.9583  2085.0
2023-06-05 04:03:00  391.2682  391.2682  391.2682  391.2682   616.0
2023-06-05 04:02:00  391.1982  391.5082  391.1982  391.2582  1327.0
2023-06-05 04:01:00  390.8483  391.1283  390.8483  391.1283  1522.0
2023-06-05 04:00:00  388.1386  390.3583  388.1386  390.3583  2974.0

[8076 rows x 5 columns]
Signals for NVDA:
               timestamp     signal cut_direction

In [ ]:
# Plot candlestick chart for GOOGLE
fig = go.Figure(data=[go.Candlestick(x=script_data['GOOGL'].index,
                                    open=script_data['GOOGL']['open'],
                                    high=script_data['GOOGL']['high'],
                                    low=script_data['GOOGL']['low'],
                                    close=script_data['GOOGL']['close'])])
fig.update_layout(title='GOOGLE Candlestick Chart')
fig.show()

In [ ]:
# Plot candlestick chart for APPLE
fig = go.Figure(data=[go.Candlestick(x=script_data['AAPL'].index,
                                    open=script_data['AAPL']['open'],
                                    high=script_data['AAPL']['high'],
                                    low=script_data['AAPL']['low'],
                                    close=script_data['AAPL']['close'])])
fig.update_layout(title='APPLE Candlestick Chart')
fig.show()

In [ ]:
# Plot candlestick chart for NVIDIA
fig = go.Figure(data=[go.Candlestick(x=script_data['NVDA'].index,
                                    open=script_data['NVDA']['open'],
                                    high=script_data['NVDA']['high'],
                                    low=script_data['NVDA']['low'],
                                    close=script_data['NVDA']['close'])])
fig.update_layout(title='NVIDIA Candlestick Chart')
fig.show()